In [2]:
library(h2o)

In [4]:
h2o.init()

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         5 seconds 370 milliseconds 
    H2O cluster timezone:       Etc/UTC 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.28.0.3 
    H2O cluster version age:    1 month and 16 days  
    H2O cluster name:           H2O_started_from_R_root_atf962 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   3.55 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         Amazon S3, XGBoost, Algos, AutoML, Core V3, TargetEncoder, Core V4 
    R Version:                  R version 3.6.0 (2019-04-26) 



In [5]:
## data url
url = "https://raw.githubusercontent.com/pandas-dev/pandas/master/pandas/tests/data/iris.csv"
df <- h2o.importFile(url)

  |======================================================================| 100%


In [6]:
head(df)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa


In [7]:
parts = h2o.splitFrame(df,0.8)

In [8]:
train <- parts[[1]]
test <- parts[[2]]

In [9]:
head(train)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa


In [10]:
head(test)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,5.1,3.5,1.4,0.2,Iris-setosa
2,5.4,3.7,1.5,0.2,Iris-setosa
3,4.3,3.0,1.1,0.1,Iris-setosa
4,5.8,4.0,1.2,0.2,Iris-setosa
5,5.1,3.5,1.4,0.3,Iris-setosa
6,5.1,3.8,1.5,0.3,Iris-setosa


In [11]:
### this is the PCA
n_PCA = h2o.prcomp(train,1:4,k=4,impute_missing = TRUE)

  |======================================================================| 100%


In [12]:
n_PCA

Model Details:

H2ODimReductionModel: pca
Model ID:  PCA_model_R_1584886216758_1 
Importance of components: 
                            pc1      pc2      pc3      pc4
Standard deviation     7.858869 1.420910 0.289218 0.157451
Proportion of Variance 0.966701 0.031601 0.001309 0.000388
Cumulative Proportion  0.966701 0.998303 0.999612 1.000000


H2ODimReductionMetrics: pca

No model metrics available for PCA



In [13]:
p_PCA = h2o.predict(n_PCA,train)

  |======================================================================| 100%


In [14]:
p_PCA

        PC1       PC2         PC3         PC4
1 -5.563743 -1.996443  0.24605493  0.09186776
2 -5.437002 -2.120978  0.01812097  0.01140680
3 -5.427639 -1.895579  0.02158866 -0.08396502
4 -5.864230 -2.356873 -0.10628411 -0.06830395
5 -6.465767 -2.357496 -0.23505305 -0.02866518
6 -5.505405 -2.098968 -0.22713111 -0.05706244

[120 rows x 4 columns] 

## now that gives you four PC1,PC2,PC3,PC4  so that means 4 columns are now congested to PC1 if you want you can throw out the other pc2,3,4 and just take PC1 for input

# GLRM
# Generalized Low Rank Models (GLRM) is an algorithm for dimensionality reduction of a dataset. It is a general, parallelized optimization algorithm that applies to a variety of loss and regularization functions

In [15]:
n_glrm = h2o.glrm(train,1:4,k=4)

  |======================================================================| 100%


In [16]:
n_glrm

Model Details:

H2ODimReductionModel: glrm
Model ID:  GLRM_model_R_1584886216758_2 
Model Summary: 
  number_of_iterations final_step_size final_objective_value
1                   74         0.00007               0.00000


H2ODimReductionMetrics: glrm
** Reported on training data. **

Sum of Squared Error (Numeric):  3.938087e-12
Misclassification Error (Categorical):  0
Number of Numeric Entries:  480
Number of Categorical Entries:  0



In [17]:
p_GLRM = h2o.predict(n_glrm,train)

  |======================================================================| 100%


In [18]:
p_GLRM

  reconstr_SepalLength reconstr_SepalWidth reconstr_PetalLength
1                  4.9                 3.0                  1.4
2                  4.7                 3.2                  1.3
3                  4.6                 3.1                  1.5
4                  5.0                 3.6                  1.4
5                  5.4                 3.9                  1.7
6                  4.6                 3.4                  1.4
  reconstr_PetalWidth
1                 0.2
2                 0.2
3                 0.2
4                 0.2
5                 0.4
6                 0.3

[120 rows x 4 columns] 

In [19]:
### so GLRM actually predict the value for you lets compare with the actual value

## we can give 5 of the column and told them to compress the data into 3 column and then predict

In [21]:
n_glrm3 = h2o.glrm(train,1:5,k=3)

  |======================================================================| 100%


In [22]:
n_glrm3

Model Details:

H2ODimReductionModel: glrm
Model ID:  GLRM_model_R_1584886216758_4 
Model Summary: 
  number_of_iterations final_step_size final_objective_value
1                  277         0.00009              11.14632


H2ODimReductionMetrics: glrm
** Reported on training data. **

Sum of Squared Error (Numeric):  11.14619
Misclassification Error (Categorical):  0
Number of Numeric Entries:  480
Number of Categorical Entries:  120



In [23]:
p_glrm3 = h2o.predict(n_glrm3,train)

  |======================================================================| 100%


In [24]:
p_glrm3

  reconstr_SepalLength reconstr_SepalWidth reconstr_PetalLength
1             4.731535            3.216412             1.460404
2             4.667617            3.240592             1.308380
3             4.594539            3.115170             1.463035
4             5.055001            3.532055             1.362057
5             5.503316            3.763105             1.672226
6             4.704823            3.264358             1.359210
  reconstr_PetalWidth reconstr_Name
1           0.2866976   Iris-setosa
2           0.2292415   Iris-setosa
3           0.3056369   Iris-setosa
4           0.2226397   Iris-setosa
5           0.3279893   Iris-setosa
6           0.2587616   Iris-setosa

[120 rows x 5 columns] 

In [28]:
pred = as.data.frame(h2o.cbind(train$Name,p_glrm3$reconstr_Name))

ERROR: Error in as_data_frame(): could not find function "as_data_frame"
